In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import scipy.spatial as spspatial

from scipy.spatial import Voronoi
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.utils import check_random_state
from tabpfn import TabPFNClassifier

In [ ]:
import numpy as np
from scipy._lib.decorator import decorator as _decorator

__all__ = ['delaunay_plot_2d', 'convex_hull_plot_2d', 'voronoi_plot_2d']


@_decorator
def _held_figure(func, obj, ax=None, **kw):
    import matplotlib.pyplot as plt

    if ax is None:
        fig = plt.figure()
        ax = fig.gca()
        return func(obj, ax=ax, **kw)

    # As of matplotlib 2.0, the "hold" mechanism is deprecated.
    # When matplotlib 1.x is no longer supported, this check can be removed.
    was_held = getattr(ax, 'ishold', lambda: True)()
    if was_held:
        return func(obj, ax=ax, **kw)
    try:
        ax.hold(True)
        return func(obj, ax=ax, **kw)
    finally:
        ax.hold(was_held)


def _adjust_bounds(ax, points):
    margin = 0.1 * points.ptp(axis=0)
    xy_min = points.min(axis=0) - margin
    xy_max = points.max(axis=0) + margin
    ax.set_xlim(xy_min[0], xy_max[0])
    ax.set_ylim(xy_min[1], xy_max[1])


@_held_figure
def delaunay_plot_2d(tri, ax=None):
    """
    Plot the given Delaunay triangulation in 2-D

    Parameters
    ----------
    tri : scipy.spatial.Delaunay instance
        Triangulation to plot
    ax : matplotlib.axes.Axes instance, optional
        Axes to plot on

    Returns
    -------
    fig : matplotlib.figure.Figure instance
        Figure for the plot

    See Also
    --------
    Delaunay
    matplotlib.pyplot.triplot

    Notes
    -----
    Requires Matplotlib.

    Examples
    --------

    >>> import numpy as np
    >>> import matplotlib.pyplot as plt
    >>> from scipy.spatial import Delaunay, delaunay_plot_2d

    The Delaunay triangulation of a set of random points:

    >>> rng = np.random.default_rng()
    >>> points = rng.random((30, 2))
    >>> tri = Delaunay(points)

    Plot it:

    >>> _ = delaunay_plot_2d(tri)
    >>> plt.show()

    """
    if tri.points.shape[1] != 2:
        raise ValueError("Delaunay triangulation is not 2-D")

    x, y = tri.points.T
    ax.plot(x, y, 'o')
    ax.triplot(x, y, tri.simplices.copy())

    _adjust_bounds(ax, tri.points)

    return ax.figure


@_held_figure
def convex_hull_plot_2d(hull, ax=None):
    """
    Plot the given convex hull diagram in 2-D

    Parameters
    ----------
    hull : scipy.spatial.ConvexHull instance
        Convex hull to plot
    ax : matplotlib.axes.Axes instance, optional
        Axes to plot on

    Returns
    -------
    fig : matplotlib.figure.Figure instance
        Figure for the plot

    See Also
    --------
    ConvexHull

    Notes
    -----
    Requires Matplotlib.


    Examples
    --------

    >>> import numpy as np
    >>> import matplotlib.pyplot as plt
    >>> from scipy.spatial import ConvexHull, convex_hull_plot_2d

    The convex hull of a random set of points:

    >>> rng = np.random.default_rng()
    >>> points = rng.random((30, 2))
    >>> hull = ConvexHull(points)

    Plot it:

    >>> _ = convex_hull_plot_2d(hull)
    >>> plt.show()

    """
    from matplotlib.collections import LineCollection

    if hull.points.shape[1] != 2:
        raise ValueError("Convex hull is not 2-D")

    ax.plot(hull.points[:, 0], hull.points[:, 1], 'o')
    line_segments = [hull.points[simplex] for simplex in hull.simplices]
    ax.add_collection(LineCollection(line_segments,
                                     colors='k',
                                     linestyle='solid'))
    _adjust_bounds(ax, hull.points)

    return ax.figure


@_held_figure
def voronoi_plot_2d(vor, ax=None, **kw):
    """
    Plot the given Voronoi diagram in 2-D

    Parameters
    ----------
    vor : scipy.spatial.Voronoi instance
        Diagram to plot
    ax : matplotlib.axes.Axes instance, optional
        Axes to plot on
    show_points : bool, optional
        Add the Voronoi points to the plot.
    show_vertices : bool, optional
        Add the Voronoi vertices to the plot.
    line_colors : string, optional
        Specifies the line color for polygon boundaries
    line_width : float, optional
        Specifies the line width for polygon boundaries
    line_alpha : float, optional
        Specifies the line alpha for polygon boundaries
    point_size : float, optional
        Specifies the size of points

    Returns
    -------
    fig : matplotlib.figure.Figure instance
        Figure for the plot

    See Also
    --------
    Voronoi

    Notes
    -----
    Requires Matplotlib.

    Examples
    --------
    >>> import numpy as np
    >>> import matplotlib.pyplot as plt
    >>> from scipy.spatial import Voronoi, voronoi_plot_2d

    Create a set of points for the example:

    >>> rng = np.random.default_rng()
    >>> points = rng.random((10,2))

    Generate the Voronoi diagram for the points:

    >>> vor = Voronoi(points)

    Use `voronoi_plot_2d` to plot the diagram:

    >>> fig = voronoi_plot_2d(vor)

    Use `voronoi_plot_2d` to plot the diagram again, with some settings
    customized:

    >>> fig = voronoi_plot_2d(vor, show_vertices=False, line_colors='orange',
    ...                       line_width=2, line_alpha=0.6, point_size=2)
    >>> plt.show()

    """
    from matplotlib.collections import LineCollection

    if vor.points.shape[1] != 2:
        raise ValueError("Voronoi diagram is not 2-D")

    if kw.get('show_points', True):
        point_size = kw.get('point_size', None)
        ax.plot(vor.points[:, 0], vor.points[:, 1], '.', markersize=point_size)
    if kw.get('show_vertices', True):
        ax.plot(vor.vertices[:, 0], vor.vertices[:, 1], 'o')

    line_colors = kw.get('line_colors', 'k')
    line_width = kw.get('line_width', 1.0)
    line_alpha = kw.get('line_alpha', 1.0)

    center = vor.points.mean(axis=0)
    ptp_bound = vor.points.ptp(axis=0)

    finite_segments = []
    infinite_segments = []
    for pointidx, simplex in zip(vor.ridge_points, vor.ridge_vertices):
        simplex = np.asarray(simplex)
        if np.all(simplex >= 0):
            finite_segments.append(vor.vertices[simplex])
        else:
            i = simplex[simplex >= 0][0]  # finite end Voronoi vertex

            t = vor.points[pointidx[1]] - vor.points[pointidx[0]]  # tangent
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # normal

            midpoint = vor.points[pointidx].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            if (vor.furthest_site):
                direction = -direction
            far_point = vor.vertices[i] + direction * ptp_bound.max()

            infinite_segments.append([vor.vertices[i], far_point])

    ax.add_collection(LineCollection(finite_segments,
                                     colors=line_colors,
                                     lw=line_width,
                                     alpha=line_alpha,
                                     linestyle='solid'))
    ax.add_collection(LineCollection(infinite_segments,
                                     colors=line_colors,
                                     lw=line_width,
                                     alpha=line_alpha,
                                     linestyle='dashed'))

    _adjust_bounds(ax, vor.points)

    return ax.figure

def my_voronoi_plot_2d(vor, ax=None, **kw):
    """
    Plot the given Voronoi diagram in 2-D

    Parameters
    ----------
    vor : scipy.spatial.Voronoi instance
        Diagram to plot
    ax : matplotlib.axes.Axes instance, optional
        Axes to plot on
    show_points : bool, optional
        Add the Voronoi points to the plot.
    show_vertices : bool, optional
        Add the Voronoi vertices to the plot.
    line_colors : string, optional
        Specifies the line color for polygon boundaries
    line_width : float, optional
        Specifies the line width for polygon boundaries
    line_alpha : float, optional
        Specifies the line alpha for polygon boundaries
    point_size : float, optional
        Specifies the size of points

    Returns
    -------
    fig : matplotlib.figure.Figure instance
        Figure for the plot

    See Also
    --------
    Voronoi

    Notes
    -----
    Requires Matplotlib.

    Examples
    --------
    >>> import numpy as np
    >>> import matplotlib.pyplot as plt
    >>> from scipy.spatial import Voronoi, voronoi_plot_2d

    Create a set of points for the example:

    >>> rng = np.random.default_rng()
    >>> points = rng.random((10,2))

    Generate the Voronoi diagram for the points:

    >>> vor = Voronoi(points)

    Use `voronoi_plot_2d` to plot the diagram:

    >>> fig = voronoi_plot_2d(vor)

    Use `voronoi_plot_2d` to plot the diagram again, with some settings
    customized:

    >>> fig = voronoi_plot_2d(vor, show_vertices=False, line_colors='orange',
    ...                       line_width=2, line_alpha=0.6, point_size=2)
    >>> plt.show()

    """
    from matplotlib.collections import LineCollection

    if vor.points.shape[1] != 2:
        raise ValueError("Voronoi diagram is not 2-D")

    if kw.get('show_points', True):
        point_size = kw.get('point_size', None)
        ax.plot(vor.points[:, 0], vor.points[:, 1], 'x', c='k', markersize=point_size)
    if kw.get('show_vertices', True):
        ax.plot(vor.vertices[:, 0], vor.vertices[:, 1], 'o')

    line_colors = kw.get('line_colors', 'k')
    line_width = kw.get('line_width', 1.0)
    line_alpha = kw.get('line_alpha', 1.0)

    center = vor.points.mean(axis=0)
    ptp_bound = vor.points.ptp(axis=0)

    finite_segments = []
    infinite_segments = []
    for pointidx, simplex in zip(vor.ridge_points, vor.ridge_vertices):
        simplex = np.asarray(simplex)
        if np.all(simplex >= 0):
            finite_segments.append(vor.vertices[simplex])
        else:
            i = simplex[simplex >= 0][0]  # finite end Voronoi vertex

            t = vor.points[pointidx[1]] - vor.points[pointidx[0]]  # tangent
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # normal

            midpoint = vor.points[pointidx].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            if (vor.furthest_site):
                direction = -direction
            far_point = vor.vertices[i] + direction * ptp_bound.max()

            infinite_segments.append([vor.vertices[i], far_point])

    ax.add_collection(LineCollection(finite_segments,
                                     colors=line_colors,
                                     lw=line_width,
                                     alpha=line_alpha,
                                     linestyle='solid'))
    ax.add_collection(LineCollection(infinite_segments,
                                     colors=line_colors,
                                     lw=line_width,
                                     alpha=line_alpha,
                                     linestyle='dashed'))

    _adjust_bounds(ax, vor.points)

    return ax.figure

In [ ]:
rng = check_random_state(42)
points = rng.random((10,2))
classifier = TabPFNClassifier(device='cpu', n_estimators=1)
X_train = points
y_train = np.array([str(_) for _ in range(10)])
classifier.fit(X_train, y_train)
x_eval = np.linspace(-0.2, 1.2, 50)
a, b = np.meshgrid(x_eval, x_eval)
X_eval = np.vstack([a.ravel(), b.ravel()]).T
#y_eval, y_prob_eval = classifier.predict(X_eval, return_winning_probability=True)
y_eval = classifier.predict(X_eval)
y_prob_eval = classifier.predict_proba(X_eval).max(axis=1)
y_pred_train = classifier.predict(X_train)

fig = plt.figure(figsize=(4,4), dpi=150);
ax = plt.gca();
vor = Voronoi(points)
fig = my_voronoi_plot_2d(
    vor, ax=ax,
    show_vertices=False, line_colors='orange',
    line_width=4, line_alpha=0.6, point_size=10)
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
for i in range(10):
    ixs_train = y_train == str(i)
    allpoints = np.vstack([X_train, X_eval])
    allpreds = np.hstack([y_pred_train, y_eval])
    ixsi = allpreds == str(i)
    plt.gca().scatter(allpoints[ixsi, 0], allpoints[ixsi, 1], color=colors[i]);
    plt.gca().scatter(X_train[ixs_train, 0], X_train[ixs_train, 1], marker='s', edgecolor='k', s=50, color=colors[i]);
plt.tight_layout();
plt.savefig("/Users/calvinm/sandbox/tabpfn-eval/v2-voronoi.png");

plt.figure();
plt.imshow(np.flipud(y_prob_eval.reshape(50, 50)))
plt.colorbar();

In [ ]:
rng = check_random_state(42)
points = rng.random((10,2))
classifier = TabPFNClassifier(device='cpu', n_estimators=32)
X_train = points
y_train = np.array([str(_) for _ in range(10)])
classifier.fit(X_train, y_train)
x_eval = np.linspace(-0.2, 1.2, 50)
a, b = np.meshgrid(x_eval, x_eval)
X_eval = np.vstack([a.ravel(), b.ravel()]).T
#y_eval, y_prob_eval = classifier.predict(X_eval, return_winning_probability=True)
y_eval = classifier.predict(X_eval)
y_prob_eval = classifier.predict_proba(X_eval).max(axis=1)
y_pred_train = classifier.predict(X_train)

fig = plt.figure(figsize=(4,4), dpi=150);
ax = plt.gca();
vor = Voronoi(points)
fig = my_voronoi_plot_2d(
    vor, ax=ax,
    show_vertices=False, line_colors='orange',
    line_width=4, line_alpha=0.6, point_size=10)
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
for i in range(10):
    ixs_train = y_train == str(i)
    allpoints = np.vstack([X_train, X_eval])
    allpreds = np.hstack([y_pred_train, y_eval])
    ixsi = allpreds == str(i)
    plt.gca().scatter(allpoints[ixsi, 0], allpoints[ixsi, 1], color=colors[i]);
    plt.gca().scatter(X_train[ixs_train, 0], X_train[ixs_train, 1], marker='s', edgecolor='k', s=50, color=colors[i]);
plt.tight_layout();
plt.savefig("/Users/calvinm/sandbox/tabpfn-eval/v2-voronoi-32.png");

plt.figure();
plt.imshow(np.flipud(y_prob_eval.reshape(50, 50)))
plt.colorbar();

In [ ]:
rng = check_random_state(42)
x_train = np.linspace(0.0, 1.0, 3)
a, b = np.meshgrid(x_train, x_train)
points = np.vstack([a.ravel(), b.ravel()]).T
X_train = points
y_train = np.array([str(_) for _ in range(9)])
x_eval = np.linspace(-0.2, 1.2, 50)
a, b = np.meshgrid(x_eval, x_eval)
X_eval = np.vstack([a.ravel(), b.ravel()]).T

classifier = TabPFNClassifier(device='cpu', n_estimators=1)
classifier.fit(X_train, y_train)
#y_eval, y_prob_eval = classifier.predict(X_eval, return_winning_probability=True)
y_eval = classifier.predict(X_eval)
y_prob_eval = classifier.predict_proba(X_eval).max(axis=1)
y_pred_train  = classifier.predict(X_train)

classifier32 = TabPFNClassifier(device='cpu', n_estimators=32)
classifier32.fit(X_train, y_train)
#y_eval32, y_prob_eval32 = classifier.predict(X_eval, return_winning_probability=True)
y_eval32 = classifier.predict(X_eval)
y_prob_eval32 = classifier.predict_proba(X_eval).max(axis=1)
y_pred_train32  = classifier.predict(X_train)

In [ ]:
fig = plt.figure(figsize=(4,4), dpi=150);
ax = plt.gca();
vor = Voronoi(points)
fig = my_voronoi_plot_2d(
    vor, ax=ax,
    show_vertices=False, line_colors='orange',
    line_width=4, line_alpha=0.6, point_size=10)
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
for i in range(10):
    ixs_train = y_train == str(i)
    allpoints = np.vstack([X_train, X_eval])
    allpreds = np.hstack([y_pred_train, y_eval])
    ixsi = allpreds == str(i)
    plt.gca().scatter(allpoints[ixsi, 0], allpoints[ixsi, 1], color=colors[i], s=25);
    plt.gca().scatter(X_train[ixs_train, 0], X_train[ixs_train, 1], marker='s', edgecolor='k', s=40, color=colors[i]);
plt.tight_layout();
plt.savefig("/Users/calvinm/sandbox/tabpfn-eval/v2-voronoi-grid.png");

plt.figure();
plt.imshow(np.flipud(y_prob_eval.reshape(50, 50)))
plt.colorbar();

In [ ]:
fig, axes = plt.subplots(figsize=(8,8), nrows=2, ncols=2, dpi=150);

rng = check_random_state(42)
points = rng.random((10,2))
classifier = TabPFNClassifier(device='cpu', n_estimators=1)
X_train = points
y_train = np.array([str(_) for _ in range(10)])
classifier.fit(X_train, y_train)
x_eval = np.linspace(-0.2, 1.2, 50)
a, b = np.meshgrid(x_eval, x_eval)
X_eval = np.vstack([a.ravel(), b.ravel()]).T
#y_eval, y_prob_eval = classifier.predict(X_eval, return_winning_probability=True)
y_eval = classifier.predict(X_eval)
y_prob_eval = classifier.predict_proba(X_eval).max(axis=1)
y_pred_train = classifier.predict(X_train)
vor = Voronoi(points)
fig = my_voronoi_plot_2d(
    vor, ax=axes[0, 0],
    show_vertices=False, line_colors='orange',
    line_width=4, line_alpha=0.6, point_size=10)
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
for i in range(10):
    ixs_train = y_train == str(i)
    allpoints = np.vstack([X_train, X_eval])
    allpreds = np.hstack([y_pred_train, y_eval])
    ixsi = allpreds == str(i)
    axes[0, 0].scatter(allpoints[ixsi, 0], allpoints[ixsi, 1], color=colors[i], s=25);
    axes[0, 0].scatter(X_train[ixs_train, 0], X_train[ixs_train, 1], marker='s', edgecolor='k', s=50, color=colors[i]);
axes[0, 0].set_xlim(-0.1, 0.9);
axes[0, 0].set_ylim(0.1, 1.1);
axes[0, 0].set_title("10 randomly-spaced points, 10 classes");
axes[0, 0].set_ylabel("No ensembling");
axes[0, 0].set_aspect("equal");

rng = check_random_state(42)
points = rng.random((10,2))
classifier = TabPFNClassifier(device='cpu', n_estimators=32)
X_train = points
y_train = np.array([str(_) for _ in range(10)])
classifier.fit(X_train, y_train)
x_eval = np.linspace(-0.2, 1.2, 50)
a, b = np.meshgrid(x_eval, x_eval)
X_eval = np.vstack([a.ravel(), b.ravel()]).T
#y_eval, y_prob_eval = classifier.predict(X_eval, return_winning_probability=True)
y_eval = classifier.predict(X_eval)
y_prob_eval = classifier.predict_proba(X_eval).max(axis=1)
y_pred_train = classifier.predict(X_train)
vor = Voronoi(points)
fig = my_voronoi_plot_2d(
    vor, ax=axes[1, 0],
    show_vertices=False, line_colors='orange',
    line_width=4, line_alpha=0.6, point_size=10)
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
for i in range(10):
    ixs_train = y_train == str(i)
    allpoints = np.vstack([X_train, X_eval])
    allpreds = np.hstack([y_pred_train, y_eval])
    ixsi = allpreds == str(i)
    axes[1, 0].scatter(allpoints[ixsi, 0], allpoints[ixsi, 1], color=colors[i], s=25);
    axes[1, 0].scatter(X_train[ixs_train, 0], X_train[ixs_train, 1], marker='s', edgecolor='k', s=50, color=colors[i]);
axes[1, 0].set_xlim(-0.1, 0.9);
axes[1, 0].set_ylim(0.1, 1.1);
axes[1, 0].set_ylabel("32 ensembles");
axes[1, 0].set_aspect("equal");

rng = check_random_state(42)
x_train = np.array([0.1, 0.5, 0.9])
a, b = np.meshgrid(x_train, x_train)
points = np.vstack([a.ravel(), b.ravel()]).T
X_train = points
y_train = np.array([str(_) for _ in range(9)])
x_eval = np.linspace(-0.2, 1.2, 50)
a, b = np.meshgrid(x_eval, x_eval)
X_eval = np.vstack([a.ravel(), b.ravel()]).T
classifier = TabPFNClassifier(device='cpu', n_estimators=1)
classifier.fit(X_train, y_train)
#y_eval, y_prob_eval = classifier.predict(X_eval, return_winning_probability=True)
y_eval = classifier.predict(X_eval)
y_prob_eval = classifier.predict_proba(X_eval).max(axis=1)
y_pred_train  = classifier.predict(X_train)
vor = Voronoi(points)
fig = my_voronoi_plot_2d(
    vor, ax=axes[0, 1],
    show_vertices=False, line_colors='orange',
    line_width=4, line_alpha=0.6, point_size=10)
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
for i in range(10):
    ixs_train = y_train == str(i)
    allpoints = np.vstack([X_train, X_eval])
    allpreds = np.hstack([y_pred_train, y_eval])
    ixsi = allpreds == str(i)
    axes[0, 1].scatter(allpoints[ixsi, 0], allpoints[ixsi, 1], color=colors[i], s=25);
    axes[0, 1].scatter(X_train[ixs_train, 0], X_train[ixs_train, 1], marker='s', edgecolor='k', s=50, color=colors[i]);
axes[0, 1].set_xlim(-0., 1.);
axes[0, 1].set_ylim(-0., 1.);    
axes[0, 1].set_title("9 grid-spaced points, 9 classes");
axes[0, 1].set_aspect("equal");

rng = check_random_state(42)
x_train = np.array([0.1, 0.5, 0.9])
a, b = np.meshgrid(x_train, x_train)
points = np.vstack([a.ravel(), b.ravel()]).T
X_train = points
y_train = np.array([str(_) for _ in range(9)])
x_eval = np.linspace(-0.2, 1.2, 50)
a, b = np.meshgrid(x_eval, x_eval)
X_eval = np.vstack([a.ravel(), b.ravel()]).T
classifier = TabPFNClassifier(device='cpu', n_estimators=32)
classifier.fit(X_train, y_train)
#y_eval, y_prob_eval = classifier.predict(X_eval, return_winning_probability=True)
y_eval = classifier.predict(X_eval)
y_prob_eval = classifier.predict_proba(X_eval).max(axis=1)
y_pred_train  = classifier.predict(X_train)
vor = Voronoi(points)
fig = my_voronoi_plot_2d(
    vor, ax=axes[1, 1],
    show_vertices=False, line_colors='orange',
    line_width=4, line_alpha=0.6, point_size=10)
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
for i in range(10):
    ixs_train = y_train == str(i)
    allpoints = np.vstack([X_train, X_eval])
    allpreds = np.hstack([y_pred_train, y_eval])
    ixsi = allpreds == str(i)
    axes[1, 1].scatter(allpoints[ixsi, 0], allpoints[ixsi, 1], color=colors[i], s=25);
    axes[1, 1].scatter(X_train[ixs_train, 0], X_train[ixs_train, 1], marker='s', edgecolor='k', s=50, color=colors[i]);
axes[1, 1].set_xlim(-0., 1.);
axes[1, 1].set_ylim(-0., 1.);
axes[1, 1].set_aspect("equal");


plt.tight_layout();
plt.savefig("/Users/calvinm/sandbox/tabpfn-eval/v2-voronois.png");